# Imports

In [4]:
import subprocess
import json
from pathlib import Path
from glob2 import glob
import os
from multiprocessing import Pool
from datetime import datetime
import pytz
from uuid import uuid4
import random
from tqdm import tqdm
import sys

# Functions

In [8]:
import subprocess
import json
from pathlib import Path
from glob2 import glob
import os
from multiprocessing import Pool
from datetime import datetime
import pytz
from uuid import uuid4
import random
from tqdm import tqdm
import sys
def get_corpus_pieces(corpus):
    return sorted (os.path.basename(d).split(".")[0] for d in glob("inputs/slices/"+corpus+"/*.csv"))

def get_corpi():
    return [os.path.basename(d) for d in glob("inputs/slices/*")][1:]

def run_command(command):
    #print(" ".join(command))    
    #res = subprocess.run(cmd, cwd="..")
    #res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")
    res = subprocess.run(command,stderr=subprocess.PIPE, cwd="..")
    #pbar.update(1)

    if res.returncode != 0:
        print("Error in subprocess:\n")
        print(res.stderr)
        return;
    
            
def run_Experiment(commands,name,iterations, threads):
    uk_tz = pytz.timezone('Europe/London')
    id = datetime.now(uk_tz).strftime('%Y%m%d%H%M%S_')+ name
    #random.shuffle(commands)

    commands = [["stack","run","fullParse","--","-n",str(iterations), "-id", str(id)] + command for command in commands]
    
    
    with Pool(threads) as p:
        for _ in tqdm(p.imap_unordered(run_command, commands), total=len(commands)*iterations):
            pass
    

        
def stochastic_res_experiment(threads=28):
    commands = []
    corpi = get_corpi()
    beamWidthRange = range (1,40,2)
    reservoirSizeRange = range(100,1000,50)
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for w in beamWidthRange:
                for r in reservoirSizeRange:
                    algo = "StochasticBeamSearch {} {}".format(w,r)
                    commands.append([ corpus, piece, algo])
    
    print("Running {} Stochastic Beam Search experiments, with beamwidth {} and reservoirSize {}".format(len(commands), beamWidthRange, reservoirSizeRange))

    run_Experiment(commands, "stochastic-all-params", threads)

def dual_stochastic_res_experiment(inputs, beamWidthRange, reservoirSizeRange, iterations=1, threads=28):
    commands = []

    for (corpus, piece) in inputs:
        for w in beamWidthRange:
            for r in reservoirSizeRange:
                algo = "DualStochasticBeamSearch {} {}".format(w,r)
                commands.append([ corpus, piece, algo])
    
    print("Running {} Stochastic Beam Search experiments, with beamwidth {} and reservoirSize {}".format(len(commands)*iterations, beamWidthRange, reservoirSizeRange))
    print("Experiment will take approx {}-{} hours\n".format(len(commands)*iterations/ 28 * 0.08, len(commands)*iterations/ 28 * 0.2))

    random.shuffle(commands)
    run_Experiment(commands, "dual-stochastic-res-params-grieg128", iterations,threads)

def all_searches_experiment(inputs, iterations=10, threads=28):
    beamWidths = range (6,36,3)
    commands = []
    for (corpus, piece) in inputs:
        algo = "DualStochasticBeamSearch {} {}".format(500,400)
        algo2 = "DualStochasticBeamSearch {} {}".format(3,1)
        commands.append([ corpus, piece, algo])
        commands.append([ corpus, piece, algo2])
    print("Running {} experiments dsbs 500 400 vs 3 1".format(len(commands)*iterations, beamWidthRange, reservoirSizeRange))
    print("Experiment will take approx {}-{} hours\n".format(len(commands)*iterations/ 28 * 0.08, len(commands)*iterations/ 28 * 0.2))

    random.shuffle(commands)
    run_Experiment(commands, "dual-stochastic-res-500-400vs3-1-grieg128", iterations,threads)

    
def stochastic_res_lim_experiment(threads=28):
    commands = []
    corpi = get_corpi()
    beamWidthRange = range (7,14,1)
    limitRange = range(1,20,1)
    reservoirSize = 700
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for w in beamWidthRange:
                for l in limitRange:
                    algo = "StochasticBeamSearchLimited {} {} {}".format(w,reservoirSize,l)
                    commands.append([ corpus, piece, algo])
    
    print("Running {} Stochastic Beam Search Limited experiments, with beamwidth {}, reservoirSize {} and spread note limit {}".format(len(commands), beamWidthRange, reservoirSize, limitRange))
    print("Experiment will take approx {}-{} hours\n".format(commands/ 28 * 0.1),commands/ 28 * 0.27)

    run_Experiment(commands, "stochastic-limited-fixed-res", threads)

In [9]:
get_corpi()

['tests', 'grieg_lyric_pieces', 'schumann_kinderszenen', 'chopin_mazurkas']

In [12]:
[("grieg_lyric_pieces", x) for x in get_corpus_pieces("grieg_lyric_pieces")]

[('grieg_lyric_pieces', 'op12n01'),
 ('grieg_lyric_pieces', 'op12n02'),
 ('grieg_lyric_pieces', 'op12n03'),
 ('grieg_lyric_pieces', 'op12n04'),
 ('grieg_lyric_pieces', 'op12n05'),
 ('grieg_lyric_pieces', 'op12n06'),
 ('grieg_lyric_pieces', 'op12n07'),
 ('grieg_lyric_pieces', 'op12n08'),
 ('grieg_lyric_pieces', 'op38n01'),
 ('grieg_lyric_pieces', 'op38n02'),
 ('grieg_lyric_pieces', 'op38n03'),
 ('grieg_lyric_pieces', 'op38n04'),
 ('grieg_lyric_pieces', 'op38n05'),
 ('grieg_lyric_pieces', 'op38n06'),
 ('grieg_lyric_pieces', 'op38n07'),
 ('grieg_lyric_pieces', 'op38n08'),
 ('grieg_lyric_pieces', 'op43n01'),
 ('grieg_lyric_pieces', 'op43n02'),
 ('grieg_lyric_pieces', 'op43n03'),
 ('grieg_lyric_pieces', 'op43n04'),
 ('grieg_lyric_pieces', 'op43n05'),
 ('grieg_lyric_pieces', 'op43n06'),
 ('grieg_lyric_pieces', 'op47n01'),
 ('grieg_lyric_pieces', 'op47n02'),
 ('grieg_lyric_pieces', 'op47n03'),
 ('grieg_lyric_pieces', 'op47n04'),
 ('grieg_lyric_pieces', 'op47n05'),
 ('grieg_lyric_pieces', 'op4

# Run Experiments

In [8]:
#if __name__ == '__main__':
#    beamWidthRange = range (10,510,50)
#    reservoirSizeRange = range(50,750,50)
#    dual_stochastic_res_experiment([("grieg_lyric_pieces", "op12n08")],beamWidthRange,reservoirSizeRange, iterations=3,threads=28)
    


Running 140 Stochastic Beam Search experiments, with beamwidth range(10, 510, 50) and reservoirSize range(50, 750, 50)
Experiment will take approx 0.4-1.0 hours



100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [1:32:03<00:00, 39.45s/it]


In [5]:
#if __name__ == '__main__':
    #algo_experiment()
    #beamwidth_experiment()

In [ ]:
if __name__ == '__main__':
    beamWidthRange = range (10,510,50)
    reservoirSizeRange = range(50,750,50)
    all_searches_experiment([("grieg_lyric_pieces", "op12n08")], iterations=1,threads=28)
    


Running 20 experiments dsbs 500 400 vs 3 1
Experiment will take approx 0.05714285714285715-0.14285714285714288 hours



  0%|                                                                                                               | 0/2 [00:00<?, ?it/s]